In [ ]:
# IGNORE THIS CELL WHICH CUSTOMIZES LAYOUT AND STYLING OF THE NOTEBOOK !
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings = lambda *a, **kw: None

# Exercise 3.2 Mesh plots (cartopy)
prepared by M.Hauser

Here we learn how to plot data as mesh grid. This is important for *gridded* model data or observations (we will introduce the interpolating function `contour` and `contourf` later). There are three functions to plot three-dimensional data in two dimensions using a colored mesh in matplotlib:

 * pcolormesh
 * pcolor
 * imshow



We will show the usage of `pcolormesh` in this exercise. This function is recommended over the others because:

 * "imshow assumes that all data elements in your array are to be rendered at the same size, whereas pcolormesh/pcolor associates elements of the data array with rectangular elements whose size may vary over the rectangular grid" (shamelessly stolen from this [stackoverflow answer](https://stackoverflow.com/a/21169703)).
 * `pcolormesh` is [about 1 to 3 orders of magnitude faster](http://thomas-cokelaer.info/blog/wp-content/uploads/2014/05/pcolor_erformance.png) than `pcolor`.

Note that most of what we show here for georeferenced plots also applies to normal usage of `pcolormesh`.

## Import libraries

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

In [ ]:
import mplotutils as mpu

## Data

We use artificial [sample data](http://scitools.org.uk/cartopy/docs/v0.15/examples/axes_grid_basic.html) to illustrate the plotting.

## First pcolormesh plot

`pcolormesh` takes x, y, z as input:

In [ ]:
# create sample data
lon, lat, data = mpu.sample_data_map(90, 48)

# ====

ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines()

ax.pcolormesh(lon, lat, data)

ax.set_global()

## Load CMIP 5 data: historical precipitation climatology (1986 to 2005)

We will load a netCDF with historical, and projected climatological precipitation, as well as the relative change between them, from all CMIP5 models for RCP8.5 (Taylor et al., 2012).

The data was prepared in [another notebook](../data/prepare_CMIP5_map.ipynb).

In [ ]:
fN = "../data/cmip5_delta_pr_rcp85_map.nc"

# load data, omitting some unnecessary variables
pr = xr.open_dataset(fN, drop_variables=["pr_rel", "proj", "agree_sign", "pval"])

pr

### Exercise
 * plot the climatological precipitation amount (`pr.hist`)

In [ ]:
# get data
lon, lat, hist = pr.lon, pr.lat, pr.hist
# ====

# plot

ax = plt.axes(projection=ccrs.Robinson())

ax.coastlines()

# code here

ax.set_global()

### Solution

In [ ]:
# get data
lon, lat, hist = pr.lon, pr.lat, pr.hist
# ====

# plot

ax = plt.axes(projection=ccrs.Robinson())

ax.coastlines()

ax.pcolormesh(lon, lat, hist, transform=ccrs.PlateCarree())

ax.set_global()

This looks all right, but what's with the white stripe?

Commonly lat and lon are in the center of the gridcell. However, by default `pcolormesh` assumes that the coordinates specify the *edges* of the gridcells and *silently truncates the topmost row and the rightmost column* in the plot!

This becomes more obvious if we have less datapoints. 

In [ ]:
# create sample data
lon, lat, data = mpu.sample_data_map(nlons=18, nlats=9)

# this is never displayed!
data[:, -1] = 5

# ====

ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines()

h = ax.pcolormesh(lon, lat, data, transform=ccrs.PlateCarree())

# plot the lat and lon data

lons, lats = np.meshgrid(lon, lat)
ax.plot(lons.flatten(), lats.flatten(), "o", transform=ccrs.PlateCarree(), ms=4, c="r")

ax.set_global()

The red points show the original lat and lon coordinates - they should be in the center of the gridcells.
Notice how there are only 8 rows and 17 columns displayed!

**NOTE**:
In the earlier versions of matplotlib this problem was remedied by passing the edges instead of the centers of the gridcells.

However in the new matplotlib pcolormesh comes with an optional argument called `shading`. 

It has options: {'flat', 'nearest', 'gouraud', 'auto'}. The default value of `shading` is 'flat'. This assumes that the coordinates specify the edges of the gridcells. In our case we need to pass 'nearest' value for the `shading` argument.


In [ ]:
# create sample data
lon, lat, data = mpu.sample_data_map(nlons=18, nlats=9)

# ====

ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines()

h = ax.pcolormesh(lon, lat, data, transform=ccrs.PlateCarree(), shading="nearest")

# plot the lat and lon data

lons, lats = np.meshgrid(lon, lat)
ax.plot(lons.flatten(), lats.flatten(), "o", transform=ccrs.PlateCarree(), ms=4, c="r")

ax.set_global()

Perfect!

### Exercise

 * apply the same correction for the cmip5 precipitation data

In [ ]:
# get data
lon, lat, hist = pr.lon, pr.lat, pr.hist

# plot

ax = plt.axes(projection=ccrs.Robinson())

ax.coastlines()

ax.pcolormesh(lon, lat, hist, transform=ccrs.PlateCarree())

ax.set_global()

### Solution

In [ ]:
# get data
lon, lat, hist = pr.lon, pr.lat, pr.hist

# plot

ax = plt.axes(projection=ccrs.Robinson())

ax.coastlines()

ax.pcolormesh(lon, lat, hist, transform=ccrs.PlateCarree(), shading="nearest")

ax.set_global()

## Saving figures // rasterized

There is nothing special about saving a map figure:

In [ ]:
# get data
lon, lat, hist = pr.lon, pr.lat, pr.hist

# plot

ax = plt.axes(projection=ccrs.Robinson())

ax.coastlines()

h = ax.pcolormesh(lon, lat, hist, transform=ccrs.PlateCarree(), shading="nearest")

ax.set_global()

plt.colorbar(h)

plt.savefig("robinson.pdf")

However, they can grow large very quickly. Especially if you save a `pcolormesh` figure as pdf, because the pdf is saved as vector graphic.

It is, however, possible to rasterize certain elements of the plot, e.g. the `pcolormesh`.

In [ ]:
# plot

ax = plt.axes(projection=ccrs.Robinson())

ax.coastlines()

h = ax.pcolormesh(
    lon, lat, hist, transform=ccrs.PlateCarree(), shading="nearest", rasterized=True
)

ax.set_global()

plt.colorbar(h)

plt.savefig("robinson_rasterized.pdf")

* Compare the file size of `'robinson_rasterized.pdf'` and `'robinson.pdf'`.

> Warning: the following line may not work in windows.

In [ ]:
! ls -lh robinson*

* Open robinson_rasterized.pdf and zoom in; you'll realise that the coastlines are not rasterized!

Let's look at a detail of the precipitation data:

In [ ]:
# plot

ax = plt.axes(projection=ccrs.Robinson())

ax.coastlines()

ax.pcolormesh(
    lon,
    lat,
    hist,
    transform=ccrs.PlateCarree(),
    shading="nearest",
    cmap="Blues",
    vmax=2500,
)

ax.set_global()

ax.set_extent([-150, -130, 30, 70], ccrs.PlateCarree())

plt.savefig("robinson_precip.pdf")

### Exercise

 * save the same plot again, but rasterize the `pcolormesh`

In [ ]:
# code here

plt.savefig("robinson_precip_rasterized.pdf")

### Solution

In [ ]:
# plot

ax = plt.axes(projection=ccrs.Robinson())

ax.coastlines()

h = ax.pcolormesh(
    lon,
    lat,
    hist,
    transform=ccrs.PlateCarree(),
    shading="nearest",
    cmap="Blues",
    vmax=2500,
    rasterized=True,
)

ax.set_global()

ax.set_extent([-150, -130, 30, 70], ccrs.PlateCarree())

plt.savefig("robinson_precip_rasterized.pdf")

## dpi

The resulting pdf does not look very good - because the rectangular elements don't have vertical edged (but the pixels do). However, savefig takes a `dpi` keyword, which saves the day.

In [ ]:
# plot

ax = plt.axes(projection=ccrs.Robinson())

ax.pcolormesh(
    lon,
    lat,
    hist,
    transform=ccrs.PlateCarree(),
    shading="nearest",
    cmap="Blues",
    vmax=2500,
    rasterized=True,
)

ax.set_extent([-150, -130, 30, 70], ccrs.PlateCarree())

plt.savefig("robinson_precip_rasterized_dpi.pdf", dpi=2000)

 * Compare the size of `'robinson_precip.pdf'`, `'robinson_precip_rasterized.pdf'`, and `'robinson_precip_rasterized_dpi.pdf'`.

> Warning: the following line may not work in windows.

In [ ]:
! ls -lh robinson_precip*

 * Open the three pdfs and compare their quality.

## Color range

We can set the range of the colors with `vmin` and `vmax`. Because we now clip values at both ends, we should let the viewers know. We can do this by usign the `extend` keyword in the colorbar. It takes the  values

 * `'neither'` (default).
 * `'both'`
 * `'min'`
 * `'max'`

**Let's open random temperature field from CESM¶**

In [ ]:
fN = '../data/cesm_temp.nc'

cesm = xr.open_dataset(fN)

cesm.temp

Let's restrict the temperature range to -30..30, and also change the colormap, using `extend='both'`.

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines()

h = ax.pcolormesh(
    cesm.lon,
    cesm.lat,
    cesm.temp - 273.15,
    transform=ccrs.PlateCarree(),
    shading= 'nearest',
    vmin=-30,
    vmax=30,
    cmap="RdBu_r",
)

ax.set_global()

plt.colorbar(h, extend="both")

### Exercise
 * clip the precipitation values to the range 0...3000
 * indicate that the values extend at the upper bound

In [ ]:
# get data
lon, lat, hist = pr.lon, pr.lat, pr.hist

# plot

ax = plt.axes(projection=ccrs.Robinson())

ax.coastlines()

h = ax.pcolormesh(lon, lat, hist, transform=ccrs.PlateCarree(), shading='nearest')

ax.set_global()

plt.colorbar(h)

### Solution

In [ ]:
# get data
lon, lat, hist = pr.lon, pr.lat, pr.hist

# plot

ax = plt.axes(projection=ccrs.Robinson())

ax.coastlines()

h = ax.pcolormesh(lon, lat, hist, transform=ccrs.PlateCarree(), vmin=0, vmax=3000, shading='nearest')

ax.set_global()

plt.colorbar(h, extend="max")

## Color levels

To create a discrete color scale instead of a continuous one, we need to pass `norm` to `pcolormesh`. `norm` is a function that normalizes data to the 0.0...1.0 interval. Usually it ranges linearly between the minimum and maximum data values. We also need to pass a changed colormap (`cmap`).

For this we can make use of a small helper function in `mpu`.

> doing this, we don't need to specify `extend` in the colorbar any more.

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines()

levels = np.arange(-30, 31, 10)
cmap, norm = mpu.from_levels_and_cmap(levels, "RdBu_r", extend="both")


h = ax.pcolormesh(
    cesm.lon, cesm.lat, cesm.temp - 273.15, transform=ccrs.PlateCarree(), norm=norm, cmap=cmap, shading='nearest'
)

ax.set_global()

plt.colorbar(h)

### Exercise

 * create discrete levels for the precipitation data
 * the colormap is called `'viridis'`

In [ ]:
# get data
lon, lat, hist = pr.lon, pr.lat, pr.hist

# plot

ax = plt.axes(projection=ccrs.Robinson())

ax.coastlines()

h = ax.pcolormesh(lon, lat, hist, transform=ccrs.PlateCarree(), vmin=0, vmax=3000, shading='nearest')

ax.set_global()

plt.colorbar(h, extend="max")

### Solution

In [ ]:
# get data
lon, lat, hist = pr.lon, pr.lat, pr.hist

# plot

ax = plt.axes(projection=ccrs.Robinson())

ax.coastlines()

levels = np.arange(0, 3001, 500)
cmap, norm = mpu.from_levels_and_cmap(levels, "viridis", extend="max")

h = ax.pcolormesh(lon, lat, hist, transform=ccrs.PlateCarree(), cmap=cmap, norm=norm, shading='nearest')

ax.set_global()

plt.colorbar(h, extend="max")

## Bonus: xarray

Until now we used xarray only as 'data store' and did the plotting as

    ax.plot(ds.lon, ds.lat. ds.data, ...)
    
However, `xarray` also has its dedicated plotting functions, which allow to do:
    
    ds.data.plot.pcolormesh(ax=ax, ...)

> It also directly takes a `levels` keyword, and applies the correct normalisation and selection of the colorbar.


In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines()

temp = cesm.temp - 273.15

levels = np.arange(-30, 31, 10)
temp.plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(), levels=levels)

ax.set_global()

### Exercise

 * plot the cmip5 precipitation data with xarray
 * specify levels

In [ ]:
# code here

### Solution

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())

ax.coastlines()

levels = np.arange(0, 3001, 500)
pr.hist.plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(), levels=levels)

ax.set_global()

### Note on the colormap

xarray includes some logic to select a colormap - if the data crosses 0 it chooses `'RdBu_r'`, else `'viridis'`, but of course you can always set it manually, with the `cmap` argument.